In [1]:
!pip install scikit-learn

In [2]:
import os
from crewai import Agent, Task, Crew
from langchain_ollama import OllamaLLM
from crewai import LLM
from sklearn.metrics import f1_score, accuracy_score


In [3]:
# Initialize LLMs
OLLAMA_BASE_URL = os.environ.get("OLLAMA_BASE_URL", "http://localhost:11434")
llama_llm_crew = LLM(
    model="ollama/llama3",
    base_url=OLLAMA_BASE_URL
)


In [4]:
llama_llm_single = OllamaLLM(base_url=OLLAMA_BASE_URL, model="llama3")

def create_agent(role, goal, backstory):
    return Agent(
        role=role,
        goal=goal,
        llm=llama_llm_crew,
        verbose=True,
        backstory=backstory
    )

# Define agents
ingredient_analyst_agent = create_agent(
    role="Ingredient Analyst",
    goal="Identify animal-based ingredients in a given recipe.",
    backstory="A meticulous food scientist with expertise in ingredient composition."
)
substitution_researcher_agent = create_agent(
    role="Substitution Researcher",
    goal="Find suitable vegan substitutes for given animal-based ingredients.",
    backstory="A knowledgeable nutritionist with access to a vast database of food ingredients."
)
recipe_adapter_agent = create_agent(
    role="Recipe Adapter",
    goal="Modify a recipe to use vegan substitutes while maintaining its overall flavor and structure.",
    backstory="A skilled chef with a deep understanding of culinary techniques and ingredient interactions."
)

# Define tasks
def create_task(description, agent, context=None, expected_output=None):
    return Task(
        description=description,
        agent=agent,
        context=context,
        expected_output=expected_output
    )

ingredient_analysis_task = create_task(
    description="Identify all animal-based ingredients in the provided recipe.",
    agent=ingredient_analyst_agent,
    expected_output="A list of animal-based ingredients."
)

substitution_research_task = create_task(
    description="Find suitable vegan substitutes for the identified animal-based ingredients. Provide a brief explanation for each substitution.",
    agent=substitution_researcher_agent,
    context=[ingredient_analysis_task],
    expected_output="A list of vegan substitutions with explanations."
)

recipe_adaptation_task = create_task(
    description="Modify the provided recipe to use the vegan substitutes. Adjust quantities and instructions as needed to ensure the recipe remains successful. Present the modified recipe in a clear, step-by-step format.",
    agent=recipe_adapter_agent,
    context=[substitution_research_task],
    expected_output="The modified vegan recipe."
)



In [5]:
def generate_vegan_recipe_multi_agent(original_recipe):
    """
    Generates a vegan version of the given recipe using a multi-agent system.

    Args:
        original_recipe: The original recipe as a string.

    Returns:
        The modified vegan recipe as a string.
    """

    ingredient_analysis_task.description = ingredient_analysis_task.description.replace("provided recipe", f"the following recipe: {original_recipe}")

    crew = Crew(
        agents=[ingredient_analyst_agent, substitution_researcher_agent, recipe_adapter_agent],
        tasks=[ingredient_analysis_task, substitution_research_task, recipe_adaptation_task],
        verbose=True
    )

    try:
        result = crew.kickoff()
        return result['recipe_adaptation_task']['output']
    except KeyError as e:
        print(f"Error: {e}")
        print("Recipe adaptation task failed. Returning original recipe.")
        return original_recipe

def generate_vegan_recipe_single_agent(original_recipe):
    """
    Generates a vegan version of the given recipe using a single LLM.

    Args:
        original_recipe: The original recipe as a string.

    Returns:
        The modified vegan recipe as a string.
    """

    prompt = f"""
    Modify the following recipe to be vegan:

    {original_recipe}

    Make sure to:
    1. Identify and replace all animal-based ingredients with suitable vegan substitutes.
    2. Adjust quantities and instructions as needed.
    3. Present the modified recipe in a clear, step-by-step format.
    """

    return llama_llm_single(prompt)

def calculate_accuracy_f1(generated_recipe, gold_standard_recipe):
    """
    Calculates accuracy and F1 score of the generated recipe.

    Args:
        generated_recipe: The generated vegan recipe.
        gold_standard_recipe: The human-written vegan recipe.

    Returns:
        A dictionary containing accuracy and F1 score.
    """

    # Tokenize the recipes
    generated_tokens = generated_recipe.lower().split()
    gold_standard_tokens = gold_standard_recipe.lower().split()

    accuracy = accuracy_score(gold_standard_tokens, generated_tokens)
    f1 = f1_score(gold_standard_tokens, generated_tokens, average='weighted')

    return {'accuracy': accuracy, 'f1_score': f1}



In [6]:
#Simple Recipe Modification

#Task: Modify a simple recipe (e.g., "Make a basic pancake recipe vegan").
#Multi-Agent Approach:
#Agent 1 (Ingredient Analysis): Identifies animal-based ingredients in the recipe.
#Agent 2 (Substitution Research): Finds suitable vegan substitutes for each identified ingredient.
#Agent 3 (Recipe Adaptation): Modifies the recipe instructions to incorporate the vegan substitutes.
#Single-Agent Approach:
#A single LLM directly modifies the recipe based on the given instruction.
#Evaluation:
#Accuracy: Check if the modified recipe is indeed vegan and whether the substitutions are appropriate and likely to result in a successful dish.
#F1 Score: Calculate F1 scores based on the overlap between the modified recipe and a human-generated vegan version of the original recipe.

original_recipe = """
Ingredients:
* 1 cup all-purpose flour
* 2 teaspoons baking powder
* 1/2 teaspoon salt
* 1/4 cup butter, melted
* 1 cup milk
* 1 egg

Instructions:
1. In a bowl, whisk together flour, baking powder, and salt.
2. In a separate bowl, whisk together melted butter, milk, and egg.
3. Gradually add the wet ingredients to the dry ingredients, mixing until just combined.
4. Heat a lightly oiled griddle or frying pan over medium heat.
5. Pour 1/4 cup batter onto the hot griddle for each pancake.
6. Cook until bubbles form on the surface and the edges are set, then flip and cook until golden brown.
"""

gold_standard_vegan_recipe = """
Ingredients:
* 1 cup all-purpose flour
* 2 teaspoons baking powder
* 1/2 teaspoon salt
* 1/4 cup plant-based butter, melted
* 1 cup plant-based milk
* 1 tablespoon ground flaxseed mixed with 3 tablespoons water (as egg replacer)

Instructions:
1. In a bowl, whisk together flour, baking powder, and salt.
2. In a separate bowl, whisk together melted plant-based butter, plant-based milk, and flaxseed mixture.
3. Gradually add the wet ingredients to the dry ingredients, mixing until just combined.
4. Heat a lightly oiled griddle or frying pan over medium heat.
5. Pour 1/4 cup batter onto the hot griddle for each pancake.
6. Cook until bubbles form on the surface and the edges are set, then flip and cook until golden brown.
"""



In [ ]:
multi_agent_recipe = generate_vegan_recipe_multi_agent(original_recipe)
single_agent_recipe = generate_vegan_recipe_single_agent(original_recipe)

multi_agent_metrics = calculate_accuracy_f1(multi_agent_recipe, gold_standard_vegan_recipe)
single_agent_metrics = calculate_accuracy_f1(single_agent_recipe, gold_standard_vegan_recipe)

print("Multi-Agent System:")
print(f"Accuracy: {multi_agent_metrics['accuracy']:.2f}")
print(f"F1 Score: {multi_agent_metrics['f1_score']:.2f}")

print("\nSingle-Agent System:")
print(f"Accuracy: {single_agent_metrics['accuracy']:.2f}")
print(f"F1 Score: {single_agent_metrics['f1_score']:.2f}")

# Agent: Ingredient Analyst
## Task: Identify all animal-based ingredients in the the following recipe: 
Ingredients:
* 1 cup all-purpose flour
* 2 teaspoons baking powder
* 1/2 teaspoon salt
* 1/4 cup butter, melted
* 1 cup milk
* 1 egg

Instructions:
1. In a bowl, whisk together flour, baking powder, and salt.
2. In a separate bowl, whisk together melted butter, milk, and egg.
3. Gradually add the wet ingredients to the dry ingredients, mixing until just combined.
4. Heat a lightly oiled griddle or frying pan over medium heat.
5. Pour 1/4 cup batter onto the hot griddle for each pancake.
6. Cook until bubbles form on the surface and the edges are set, then flip and cook until golden brown.
.


# Agent: Ingredient Analyst
## Final Answer: 
* Egg


# Agent: Substitution Researcher
## Task: Find suitable vegan substitutes for the identified animal-based ingredients. Provide a brief explanation for each substitution.
